In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5
print(inputs)

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
print(hidden)
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    print(i, i.view(1, 1, -1).shape)
    out, hidden = lstm(i.view(1, 1, -1), hidden)
    print(out)
    print(hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
print(inputs.shape)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

[tensor([[-0.5525,  0.6355, -0.3968]]), tensor([[-0.6571, -1.6428,  0.9803]]), tensor([[-0.0421, -0.8206,  0.3133]]), tensor([[-1.1352,  0.3773, -0.2824]]), tensor([[-2.5667, -1.4303,  0.5009]])]
(tensor([[[ 0.5438, -0.4057,  1.1341]]]), tensor([[[-1.1115,  0.3501, -0.7703]]]))
tensor([[-0.5525,  0.6355, -0.3968]]) torch.Size([1, 1, 3])
tensor([[[-0.2682,  0.0304, -0.1526]]], grad_fn=<StackBackward>)
(tensor([[[-0.2682,  0.0304, -0.1526]]], grad_fn=<StackBackward>), tensor([[[-1.0766,  0.0972, -0.5498]]], grad_fn=<StackBackward>))
tensor([[-0.6571, -1.6428,  0.9803]]) torch.Size([1, 1, 3])
tensor([[[-0.5370,  0.0346, -0.1958]]], grad_fn=<StackBackward>)
(tensor([[[-0.5370,  0.0346, -0.1958]]], grad_fn=<StackBackward>), tensor([[[-1.1552,  0.1214, -0.2974]]], grad_fn=<StackBackward>))
tensor([[-0.0421, -0.8206,  0.3133]]) torch.Size([1, 1, 3])
tensor([[[-0.3947,  0.0391, -0.1217]]], grad_fn=<StackBackward>)
(tensor([[[-0.3947,  0.0391, -0.1217]]], grad_fn=<StackBackward>), tensor([[[-1.

In [4]:
def prepare_sequence(seq, word_to_ix):
    sen_tensor = [word_to_ix[w] for w in seq]
    sen_tensor = torch.tensor(sen_tensor, dtype=torch.long)
    word_lis = [torch.tensor(convert_word(w), dtype=torch.long) for w in seq]
    # print(word_lis, seq, sen_tensor)
    return sen_tensor, word_lis

def convert_word(word):
    lis = [ord(w)-ord('a') for w in word]
    return lis

training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("the dog ate the apple".lower().split(), ["DET", "NN", "V", "DET", "NN"]),
    ("everybody read that book".lower().split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'the': 0, 'dog': 1, 'ate': 2, 'apple': 3, 'everybody': 4, 'read': 5, 'that': 6, 'book': 7}


In [162]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        print("shape ",embeds.view(len(sentence), 1, -1).shape)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [167]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs,_ = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in,_ = prepare_sequence(sentence, word_to_ix)
        targets,_ = prepare_sequence(tags, tag_to_ix)
        print(sentence_in)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
        print(tag_scores.shape)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs,_ = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

shape  torch.Size([5, 1, 6])
tensor([[-1.2213, -0.8799, -1.2367],
        [-1.2144, -0.8608, -1.2719],
        [-1.1996, -0.8135, -1.3650],
        [-1.2004, -0.8721, -1.2699],
        [-1.2349, -0.8861, -1.2145]])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([

torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4

tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1,

tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5, 6, 7])
shape  torch.Size([4, 1, 6])
torch.Size([4, 3])
tensor([0, 1, 2, 0, 3])
shape  torch.Size([5, 1, 6])
torch.Size([5, 3])
tensor([4, 5,

In [44]:
class CharacterLevelPosTagger(nn.Module):

    def __init__(self, embedding_dim_charlevel, hidden_dim_char_level, embedding_dim_wordlevel, hidden_dim_word_level, vocab_size, targset_size):
        super(CharacterLevelPosTagger, self).__init__()
        
        # character level LSTM 
        self.char_embeddings = nn.Embedding(26, embedding_dim_charlevel)
        self.lstm_charlevel = nn.LSTM(embedding_dim_charlevel, hidden_dim_char_level)
        
        # pos tagger LSTM
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim_wordlevel)
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm_wordlevel = nn.LSTM(embedding_dim_wordlevel+hidden_dim_char_level, hidden_dim_word_level)
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim_word_level, targset_size)
        self.target_size = targset_size

    def forward(self, sentence, char_in):
        cw_vec = []
        for ch in char_in:
            embeds = self.char_embeddings(ch)
            # print(ch, embeds)
            for i in range(0, len(embeds)):
                temp = embeds[i].view(1, 1, len(embeds[i]))
                #print(temp.shape)
                out, hidden = self.lstm_charlevel(temp)
                #print(out.shape, hidden)
                #print(out[-1])
            # print(out[-1])
            cw_vec.append(out[-1]) # this word's final character level representation 
        w_embeds = self.word_embeddings(sentence)
        # print(len(cw_vec), len(w_embeds))
        # print(cw_vec)
        tag_scores_tensor = torch.Tensor(1, len(sentence), self.target_size)
        tag_scores_vec = []
        for i in range(0, len(w_embeds)):
            # print(w_embeds[i].shape, cw_vec[i][0].shape)
            # print(w_embeds[i], cw_vec[i][0])
            inp = torch.cat((cw_vec[i][0], w_embeds[i]), 0)
            # print(inp.shape)
            out, hidden = self.lstm_wordlevel(inp.view(1, 1, len(inp)))
            tag_space = self.hidden2tag(out[-1])
            # print(tag_space)
            tag_scores = F.log_softmax(tag_space, dim=1)
            tag_scores_vec.append(torch.squeeze(tag_scores))
            # print(tag_scores)
        a = torch.stack(tag_scores_vec)
        print(a.shape)
        return a

In [45]:
model = CharacterLevelPosTagger(embedding_dim_charlevel=5, 
                                hidden_dim_char_level=3, 
                                embedding_dim_wordlevel=5, 
                                hidden_dim_word_level=3,
                                vocab_size=len(word_to_ix), targset_size=len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [51]:


# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
"""
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)
"""

for epoch in range(50):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in, char_in = prepare_sequence(sentence, word_to_ix)
        targets,_ = prepare_sequence(tags, tag_to_ix)
        print(sentence_in, char_in, targets)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in, char_in)
        print(tag_scores)
        
        
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
        

# See what the scores are after training
with torch.no_grad():
    inputs,char_in = prepare_sequence(training_data[1][0], word_to_ix)
    tag_scores = model(inputs, char_in)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)
print('shesh')

tensor([0, 1, 2, 0, 3]) [tensor([19,  7,  4]), tensor([ 3, 14,  6]), tensor([ 0, 19,  4]), tensor([19,  7,  4]), tensor([ 0, 15, 15, 11,  4])] tensor([0, 1, 2, 0, 1])
torch.Size([5, 3])
tensor([[-0.1626, -2.0763, -3.7009],
        [-3.4347, -0.0862, -2.9893],
        [-2.6330, -2.9257, -0.1341],
        [-0.1626, -2.0763, -3.7009],
        [-0.9607, -0.6184, -2.5438]], grad_fn=<StackBackward>)
tensor([4, 5, 6, 7]) [tensor([ 4, 21,  4, 17, 24,  1, 14,  3, 24]), tensor([17,  4,  0,  3]), tensor([19,  7,  0, 19]), tensor([ 1, 14, 14, 10])] tensor([1, 2, 0, 1])
torch.Size([4, 3])
tensor([[-3.3174, -0.0934, -2.9393],
        [-3.1558, -1.8494, -0.2231],
        [-0.1878, -1.9781, -3.4142],
        [-3.1251, -0.0676, -3.8410]], grad_fn=<StackBackward>)
tensor([0, 1, 2, 0, 3]) [tensor([19,  7,  4]), tensor([ 3, 14,  6]), tensor([ 0, 19,  4]), tensor([19,  7,  4]), tensor([ 0, 15, 15, 11,  4])] tensor([0, 1, 2, 0, 1])
torch.Size([5, 3])
tensor([[-0.1611, -2.0849, -3.7109],
        [-3.4494, -0

torch.Size([4, 3])
tensor([[-3.5766, -0.0711, -3.2020],
        [-3.3849, -2.0036, -0.1848],
        [-0.1590, -2.1257, -3.5893],
        [-3.4063, -0.0506, -4.1267]], grad_fn=<StackBackward>)
tensor([0, 1, 2, 0, 3]) [tensor([19,  7,  4]), tensor([ 3, 14,  6]), tensor([ 0, 19,  4]), tensor([19,  7,  4]), tensor([ 0, 15, 15, 11,  4])] tensor([0, 1, 2, 0, 1])
torch.Size([5, 3])
tensor([[-0.1363, -2.2362, -3.8859],
        [-3.7144, -0.0636, -3.2908],
        [-2.7710, -3.2157, -0.1084],
        [-0.1363, -2.2362, -3.8859],
        [-1.0427, -0.5508, -2.6444]], grad_fn=<StackBackward>)
tensor([4, 5, 6, 7]) [tensor([ 4, 21,  4, 17, 24,  1, 14,  3, 24]), tensor([17,  4,  0,  3]), tensor([19,  7,  0, 19]), tensor([ 1, 14, 14, 10])] tensor([1, 2, 0, 1])
torch.Size([4, 3])
tensor([[-3.5895, -0.0702, -3.2151],
        [-3.3961, -2.0112, -0.1831],
        [-0.1577, -2.1327, -3.5977],
        [-3.4203, -0.0498, -4.1407]], grad_fn=<StackBackward>)
tensor([0, 1, 2, 0, 3]) [tensor([19,  7,  4]), ten

tensor([0, 1, 2, 0, 3]) [tensor([19,  7,  4]), tensor([ 3, 14,  6]), tensor([ 0, 19,  4]), tensor([19,  7,  4]), tensor([ 0, 15, 15, 11,  4])] tensor([0, 1, 2, 0, 1])
torch.Size([5, 3])
tensor([[-0.1177, -2.3699, -4.0435],
        [-3.9669, -0.0487, -3.5527],
        [-2.9001, -3.4622, -0.0903],
        [-0.1177, -2.3699, -4.0435],
        [-1.1142, -0.4990, -2.7382]], grad_fn=<StackBackward>)
tensor([4, 5, 6, 7]) [tensor([ 4, 21,  4, 17, 24,  1, 14,  3, 24]), tensor([17,  4,  0,  3]), tensor([19,  7,  0, 19]), tensor([ 1, 14, 14, 10])] tensor([1, 2, 0, 1])
torch.Size([4, 3])
tensor([[-3.8358, -0.0544, -3.4636],
        [-3.6066, -2.1522, -0.1548],
        [-0.1366, -2.2613, -3.7536],
        [-3.6848, -0.0381, -4.4015]], grad_fn=<StackBackward>)
tensor([0, 1, 2, 0, 3]) [tensor([19,  7,  4]), tensor([ 3, 14,  6]), tensor([ 0, 19,  4]), tensor([19,  7,  4]), tensor([ 0, 15, 15, 11,  4])] tensor([0, 1, 2, 0, 1])
torch.Size([5, 3])
tensor([[-0.1169, -2.3760, -4.0507],
        [-3.9789, -0

In [9]:
a = []
for i in range(100):
    a.append(torch.rand(1, 100, 100))

In [10]:
b = torch.Tensor(100, 100, 100)
torch.cat(a, out=b)

tensor([[[0.6341, 0.9883, 0.7677,  ..., 0.2461, 0.8834, 0.3537],
         [0.7293, 0.5456, 0.1510,  ..., 0.3509, 0.3768, 0.4817],
         [0.1153, 0.2992, 0.9669,  ..., 0.2666, 0.9228, 0.8555],
         ...,
         [0.4615, 0.3764, 0.9054,  ..., 0.0757, 0.3366, 0.8268],
         [0.4457, 0.0902, 0.9996,  ..., 0.1675, 0.6444, 0.7142],
         [0.3792, 0.1043, 0.4753,  ..., 0.2308, 0.7309, 0.5620]],

        [[0.4988, 0.3917, 0.4446,  ..., 0.5313, 0.4872, 0.4695],
         [0.1782, 0.3542, 0.2654,  ..., 0.7713, 0.2221, 0.9433],
         [0.2613, 0.0685, 0.9109,  ..., 0.8270, 0.6558, 0.3328],
         ...,
         [0.6575, 0.6190, 0.3706,  ..., 0.5811, 0.5628, 0.0277],
         [0.5957, 0.0603, 0.4324,  ..., 0.1442, 0.0731, 0.8496],
         [0.5991, 0.4288, 0.2107,  ..., 0.1643, 0.3512, 0.6955]],

        [[0.7707, 0.5431, 0.4706,  ..., 0.4531, 0.0493, 0.8663],
         [0.5422, 0.1269, 0.1365,  ..., 0.3603, 0.1764, 0.6325],
         [0.6196, 0.3060, 0.4548,  ..., 0.0535, 0.4200, 0.

In [11]:
print(b.shape)

torch.Size([100, 100, 100])
